<a href="https://colab.research.google.com/github/jattFaizan423/Faizan-jatt/blob/main/New_sale_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
sale = pd.read_csv('/content/sales.csv')

In [57]:
pd.set_option('display.max_columns',None)

In [58]:
sale.head()

,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,category,payment_method,bi_st,cust_id,year,month,ref_num,Name Prefix,First Name,Middle Initial,Last Name,Gender,age,full_name,E Mail,Customer Since,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
0,100354678,01/10/2020,received,574772,oasis_Oasis-064-36,21.0,89.9,1798.0,0.0,1798.0,Men's Fashion,cod,Valid,60124.0,2020.0,Oct-20,987867.0,Drs.,Jani,W,Titus,F,43.0,"Titus, Jani",jani.titus@gmail.com,8/22/2006,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571.0,South,jwtitus,0.0
1,100354678,01/10/2020,received,574774,Fantastic_FT-48,11.0,19.0,190.0,0.0,190.0,Men's Fashion,cod,Valid,60124.0,2020.0,Oct-20,987867.0,Drs.,Jani,W,Titus,F,43.0,"Titus, Jani",jani.titus@gmail.com,8/22/2006,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571.0,South,jwtitus,0.0
2,100354680,01/10/2020,complete,574777,mdeal_DMC-610-8,9.0,149.9,1199.2,0.0,1199.2,Men's Fashion,cod,Net,60124.0,2020.0,Oct-20,987867.0,Drs.,Jani,W,Titus,F,43.0,"Titus, Jani",jani.titus@gmail.com,8/22/2006,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571.0,South,jwtitus,0.0
3,100354680,01/10/2020,complete,574779,oasis_Oasis-061-36,9.0,79.9,639.2,0.0,639.2,Men's Fashion,cod,Net,60124.0,2020.0,Oct-20,987867.0,Drs.,Jani,W,Titus,F,43.0,"Titus, Jani",jani.titus@gmail.com,8/22/2006,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571.0,South,jwtitus,0.0
4,100367357,13/11/2020,received,595185,MEFNAR59C38B6CA08CD,2.0,99.9,99.9,0.0,99.9,Men's Fashion,cod,Valid,60124.0,2020.0,Nov-20,987867.0,Drs.,Jani,W,Titus,F,43.0,"Titus, Jani",jani.titus@gmail.com,8/22/2006,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571.0,South,jwtitus,0.0


In [4]:
subset = sale.loc[0:20000,:]

In [5]:
df = subset.copy()

In [6]:
pd.set_option('display.max_rows',None)

In [7]:
pd.set_option('display.max_columns',None)

In [8]:
df['order_id'] = df['order_id'].astype('int64')

In [9]:
df['order_date'] = pd.to_datetime(df['order_date'], dayfirst=True)

In [10]:
df['status'] = df['status'].str.strip().str.lower().astype('category')

In [11]:
df['item_id'] = df['item_id'].astype('int64')

In [12]:
df['qty_ordered'] = df['qty_ordered'].astype('int32')

In [13]:
df['price'] = df['price'].astype(float)

In [14]:
df.rename({'value':'pre_discount_total'},axis=1,inplace=True)

In [15]:
df['pre_discount_total'] = df['pre_discount_total'].astype(float)

In [16]:
# Step 1: Calculate full price (without discount)
df['full_price'] = df['qty_ordered'] * df['price']

# Step 2: Calculate discount as full_price - value
df['calculated_discount'] = df['full_price'] - df['pre_discount_total']

# Step 3: Only update discount_amount where it's 0
df.loc[df['discount_amount'] == 0, 'discount_amount'] = df.loc[df['discount_amount'] == 0, 'calculated_discount']

# Optional: Drop helper column
df.drop(columns=['full_price', 'calculated_discount'], inplace=True)

In [17]:
df['discount_amount'] = df['discount_amount'].astype(float)

In [18]:
df['total'] = df['total'].astype(float)

In [19]:
df['total'] = df['total'].round(1)

In [20]:
df['category'] = df['category'].str.strip().str.replace('&',' ').str.lower().astype('category')

In [21]:
df['payment_method'] = df['payment_method'].str.strip().str.replace('_',' ').str.lower().astype('category')

In [22]:
df.rename(columns={'bi_st': 'sale_status'},inplace=True)

In [23]:
def standardize_sale_status(status):
    status = status.lower()  # convert to lowercase for case-insensitive comparison
    if status == 'valid':
        return 'Completed'
    elif status == 'groos':
        return 'Total Amount'
    elif status == 'net':
        return 'Final Amount'
    else:
        return status  # return original if no match found

# Example usage:
df['sale_status'] = df['sale_status'].apply(standardize_sale_status)

In [24]:
df['sale_status'] = df['sale_status'].str.strip().str.lower().astype('category')

In [25]:
df['cust_id'] = df['cust_id'].astype('int64')

In [26]:
df['year'] = df['year'].astype('category')

In [27]:
date = df['month'].str.strip().str.split('-').str[1]

In [28]:
df.insert(16,'date',date)

In [29]:
df['date'] = df['date'].astype('int32')

In [30]:
df['month'] = df['month'].str.strip().str.split('-').str[0]

In [31]:
df['month'] = df['month'].str.strip().str.lower().astype('category')

In [32]:
df['ref_num'] = df['ref_num'].astype('int64')

In [33]:
df['Name Prefix'] = df['Name Prefix'].str.strip().str.replace('.','').str.lower().astype('category')

In [34]:
df['full_name'] = df['full_name'].str.strip().str.replace(',',' ').str.lower().astype('category')

In [35]:
def categorize_gender(gender_code):

    if gender_code == 'F':
        return 'Female'
    elif gender_code == 'M':
        return 'Male'
    else:
        return 'Unknown'
df['Gender'] = df['Gender'].apply(categorize_gender)

In [36]:
df['Gender'] = df['Gender'].str.strip().str.lower().astype('category')

In [37]:
df['age'] = df['age'].astype('int32')

In [38]:
df['Customer Since'] = pd.to_datetime(df['Customer Since'])

In [39]:
df['SSN'] = df['SSN'].str.strip().str.replace('-','').astype('category')

In [40]:
df.rename(columns={'SSN':'social_security_number'},inplace=True)

In [41]:
# df['Phone No. '].str.replace('-',' ').value_counts()
!pip install phonenumbers
import phonenumbers
def format_e164(phone):
    try:
        parsed = phonenumbers.parse(phone, "US")
        return phonenumbers.format_number(parsed, phonenumbers.PhoneNumberFormat.E164)
    except:
        return None
df['Phone No. '] = df['Phone No. '].apply(format_e164)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.7 MB/s eta 0:00:00


In [42]:
df['Phone No. '] = df['Phone No. '].astype('category')

In [43]:
df['Place Name'] = df['Place Name'].str.strip().str.lower().astype('category')

In [44]:
df['County'] = df['County'].str.strip().str.lower().astype('category')

In [45]:
df['City'] = df['City'].str.strip().str.lower().astype('category')

In [46]:
def get_state_full_name(abbreviation):
    state_mapping = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
        'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
        'DC': 'District of Columbia', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii',
        'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
        'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine',
        'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota',
        'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska',
        'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
        'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
        'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
        'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas',
        'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington',
        'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
    }
    return state_mapping.get(abbreviation.upper(), abbreviation)

df['State'] = df['State'].apply(get_state_full_name)

In [47]:
df['State'] = df['State'].str.strip().str.lower().astype('category')

In [48]:
df['Region'] = df['Region'].str.strip().str.lower().astype('category')

In [49]:
df['User Name'] = df['User Name'].str.strip().str.lower().astype('category')

In [50]:
df['discount_percentage'] = (df['discount_amount'] / df['pre_discount_total']) * 100

In [51]:
df['E Mail'] = df['E Mail'].astype('category')

In [52]:
df.drop(columns=['sku','First Name','Middle Initial','Last Name','Zip','Discount_Percent'],inplace=True)

In [53]:
df['discount_amount'] = df['discount_amount'].fillna(200.00000)

In [60]:
df['year'] = df['year'].astype('int32')

In [54]:
# df.to_excel("new_sales.xlsx", index=False, sheet_name="new sale_data")

In [61]:
df.head(10)

,order_id,order_date,status,item_id,qty_ordered,price,pre_discount_total,discount_amount,total,category,payment_method,sale_status,cust_id,year,month,date,ref_num,Name Prefix,Gender,age,full_name,E Mail,Customer Since,social_security_number,Phone No.,Place Name,County,City,State,Region,User Name,discount_percentage
0,100354678,2020-10-01,received,574772,21,89.9,1798.0,89.9,1798.0,men's fashion,cod,completed,60124,2020,oct,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,5.0
1,100354678,2020-10-01,received,574774,11,19.0,190.0,19.0,190.0,men's fashion,cod,completed,60124,2020,oct,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,10.0
2,100354680,2020-10-01,complete,574777,9,149.9,1199.2,149.9,1199.2,men's fashion,cod,final amount,60124,2020,oct,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,12.5
3,100354680,2020-10-01,complete,574779,9,79.9,639.2,79.9,639.2,men's fashion,cod,final amount,60124,2020,oct,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,12.5
4,100367357,2020-11-13,received,595185,2,99.9,99.9,99.9,99.9,men's fashion,cod,completed,60124,2020,nov,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,100.0
5,100367357,2020-11-13,received,595186,2,39.9,39.9,39.9,39.9,men's fashion,cod,completed,60124,2020,nov,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,100.0
6,100367360,2020-11-13,order_refunded,595192,2,47.6,47.6,47.6,47.6,mobiles tablets,cod,completed,60124,2020,nov,20,987867,drs,female,43,titus jani,jani.titus@gmail.com,2006-08-22,627315251,+14059591129,vinson,harmon,vinson,oklahoma,south,jwtitus,100.0
7,100354677,2020-10-01,canceled,574769,2,49.0,49.0,49.0,49.0,mobiles tablets,payaxis,gross,42485,2020,oct,20,171143,prof,male,28,eaker lee,lee.eaker@gmail.com,1981-02-04,185864345,+12393356755,graham,bradford,graham,florida,south,lseaker,100.0
8,100354677,2020-10-01,canceled,574770,2,135.0,135.0,135.0,135.0,computing,payaxis,gross,42485,2020,oct,20,171143,prof,male,28,eaker lee,lee.eaker@gmail.com,1981-02-04,185864345,+12393356755,graham,bradford,graham,florida,south,lseaker,100.0
9,100354677,2020-10-01,canceled,574771,2,549.9,549.9,549.9,549.9,appliances,payaxis,gross,42485,2020,oct,20,171143,prof,male,28,eaker lee,lee.eaker@gmail.com,1981-02-04,185864345,+12393356755,graham,bradford,graham,florida,south,lseaker,100.0


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20001 entries, 0 to 20000
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   order_id                20001 non-null  int64         
 1   order_date              20001 non-null  datetime64[ns]
 2   status                  20001 non-null  category      
 3   item_id                 20001 non-null  int64         
 4   qty_ordered             20001 non-null  int32         
 5   price                   20001 non-null  float64       
 6   pre_discount_total      20001 non-null  float64       
 7   discount_amount         20001 non-null  float64       
 8   total                   20001 non-null  float64       
 9   category                20001 non-null  category      
 10  payment_method          20001 non-null  category      
 11  sale_status             20001 non-null  category      
 12  cust_id                 20001 non-null  int64 